In [ ]:
from api.plural import PluralSight
import pandas as pd
import json

In [ ]:


# load the JSON data
with open('data2.json') as f:
    data = json.load(f)

# filter the data for model type "users.user"
user_data = [d for d in data if d['model'] == 'users.user']

# filter the data for model type "portfolio.portfolio"
portfolio_data = [d for d in data if d['model'] == 'portfolio.portfolio']

# convert the filtered JSON data to a data frame
codershq_users_df = pd.json_normalize(user_data)
codershq_portfolio_df = pd.json_normalize(portfolio_data)

# remove the "fields." prefix from column names
codershq_users_df.columns = codershq_users_df.columns.str.replace('fields\.', '', regex=True)
codershq_portfolio_df.columns = codershq_portfolio_df.columns.str.replace('fields\.', '', regex=True)

codershq_merged_df = codershq_users_df.set_index('pk').combine_first(codershq_portfolio_df.set_index('user')).reset_index()



In [ ]:
pluralsight = PluralSight()
plural_all_skills = pluralsight.all_skills()
plural_all_users = pluralsight.all_p_users()

plural_all_skills_df = pd.DataFrame(plural_all_skills)
plural_all_users_df = pd.DataFrame(plural_all_users)

# merge the data frames on the "userId" and "id" columns
plural_merged_df = pd.merge(plural_all_skills_df, plural_all_users_df, left_on='userId', right_on='id')

# rename the merged column "userId" to "id"
plural_merged_df = plural_merged_df.rename(columns={'userId': 'pluralId'})

# drop rows with null values in "pluralSightEmail" column of codershq_users_df
codershq_merged_df = codershq_merged_df.dropna(subset=["pluralSightEmail"])

# merge the data frames on "email" and "pluralSightEmail" columns
merged_df = pd.merge(plural_merged_df, codershq_merged_df, left_on="email", right_on="pluralSightEmail")

# print the merged data frame
print(merged_df)


merged_df.to_csv("data/all_data.csv")